In [1]:
#Import Dependencies
import pandas as pd
import os

In [2]:
#Create filepaths for csv files with school data
school_data_file = "Resources/schools_complete.csv"
#school_data_file = os.path.join("Resources","school_complete.csv")
student_data_file = "Resources/students_complete.csv"
#student_data_file = os.path.join("Resources","students_complete.csv")

In [3]:
#Read school and student data into DataFrames
school_df = pd.read_csv(school_data_file)
student_df = pd.read_csv(student_data_file)


In [4]:
#Merge DataFrames to complete analysis
students_schools_complete = pd.merge(student_df,school_df,how="left",on="school_name")
students_schools_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [5]:
#Calculate # of schools
total_schools =len(students_schools_complete["school_name"].unique())
print(total_schools)

15


In [6]:
#Calculate # of students
total_students= students_schools_complete["Student ID"].count()
print(total_students)

39170


In [15]:
#Calculate district budget
total_budget = school_df['budget'].sum()
print(total_budget)


24649428


In [16]:
#Calculate average math score of all students
avg_math_score = students_schools_complete['math_score'].mean()
print(avg_math_score)


78.98537145774827


In [17]:
#Calculate average reading score of all students
avg_reading_score = students_schools_complete['reading_score'].mean()
print(avg_reading_score)

81.87784018381414


In [18]:
#Calculate % of students with a passing math score
passing_math_count = students_schools_complete[(students_schools_complete["math_score"] >= 70)].count()["student_name"]
percent_passing_math = passing_math_count / float(total_students) * 100
print(percent_passing_math)


74.9808526933878


In [20]:
#Calculate % of students with a passing reading score
passing_reading_count = students_schools_complete[(students_schools_complete["reading_score"] >= 70)].count()["student_name"]
percent_passing_reading = passing_reading_count / float(total_students) * 100
print(percent_passing_reading)


85.80546336482001


In [22]:
#Calculate % of students with passing scores for both math and reading
passing_math_reading_count = students_schools_complete[
    (students_schools_complete["math_score"] >= 70) & (students_schools_complete["reading_score"] >= 70)
].count()["student_name"]
percent_passing_overall = passing_math_reading_count / float(total_students) *100
print(percent_passing_overall) 

65.17232575950983


In [26]:
#Create a DataFrame to display the district's key metrics
district_summary = pd.DataFrame({"Total Schools":[total_schools],"Total Students":[total_students]
, "Total Budget":[total_budget], "Average Math Score":[avg_math_score],"Average Reading Score":[avg_reading_score]
, "% Passing Math":[percent_passing_math],"% Passing Reading":[percent_passing_reading],"% Overall Passing":[percent_passing_overall]})

# Formatting
#district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
#district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
#district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}".format)
#district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}".format) 
#district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
#district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format) 
#district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}%".format)   

#district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [ ]:
school_name = school_df["school_name"]

In [ ]:
school_types = school_df.set_index(["school_name"])["type"]

In [ ]:
per_school_students = students_schools_complete.groupby(["school_name"]).count()["Student ID"]


In [ ]:
per_school_budget = students_schools_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_students